In [2]:
from torch.utils.data import Dataset, DataLoader
import torch
from torchvision import transforms
import numpy as np
import h5py

In [18]:
def div(field):
    Fx_x = np.gradient(field[0], axis=0)
    Fy_y = np.gradient(field[1], axis=1)
    div = np.stack([Fx_x, Fy_y], axis=0)

    return div.sum(axis=0)

def div_torch(field):
    Fx_x = torch.gradient(field[0], dim=0)
    Fy_y = torch.gradient(field[1], dim=1)

    div = torch.stack([Fx_x[0], Fy_y[0]], dim=0)

    return torch.sum(div, dim=0)

In [40]:
class MagnetismData(Dataset):
    def __init__(self, db, transform=None):
        self.db = db
        self.transform = transform

    def __len__(self):
        return self.db['field'].shape[0]

    def __getitem__(self, idx):
        field = self.db['field'][idx].transpose(1,2,0)
        #print(abs(div(self.db['field'][idx])).mean())
        if self.transform:
            field = self.transform(field)
        return field

In [4]:
magnetdb = h5py.File('/home/s214435/data/magfield_symm_64.h5')
dbstd = np.std(magnetdb['field'])
channels = magnetdb["field"].shape[1]

In [59]:
tf = transforms.Compose([
        transforms.ToTensor(), transforms.Normalize((0.0, )*channels, (dbstd, )*channels)
    ])

dataset = MagnetismData(
magnetdb,
transform=tf
)

dataloader = DataLoader(dataset, batch_size=4, shuffle=False, num_workers=1)

In [75]:
#print(dataset[0].numpy().shape)
print(abs(div(magnetdb["field"][0])).mean())
print(abs(div(dataset[0].numpy()*dbstd)).mean())
print(torch.mean(torch.abs(div_torch(dataset[0]*dbstd))))
#print(magnetdb["field"][0])
#print(dataset[0].numpy())
print(np.array_equal(magnetdb["field"][0], dataset[0].numpy()*dbstd))

4.757722e-06
4.7577078e-06
tensor(4.7577e-06)
False


In [87]:
samples=next(iter(dataloader)).requires_grad_(True)
samples.shape

torch.Size([4, 2, 64, 64])

In [88]:
for sam in samples:
    print(torch.mean(torch.abs(div_torch(sam*dbstd))))

tensor(4.7577e-06, grad_fn=<MeanBackward0>)
tensor(5.3377e-06, grad_fn=<MeanBackward0>)
tensor(2.8505e-06, grad_fn=<MeanBackward0>)
tensor(3.2964e-06, grad_fn=<MeanBackward0>)


In [90]:
out = torch.sum(torch.vmap(lambda x: torch.mean(torch.abs(div_torch(x*dbstd))))(samples))

In [91]:
out.backward()

In [93]:
samples.grad[0]

tensor([[[-5.4711e-05,  5.4711e-05,  5.4711e-05,  ..., -1.8237e-05,
          -1.8237e-05, -1.8237e-05],
         [ 1.8237e-05, -1.8237e-05, -1.8237e-05,  ...,  5.4711e-05,
           5.4711e-05,  5.4711e-05],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00],
         ...,
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00],
         [ 5.4711e-05,  5.4711e-05,  5.4711e-05,  ..., -5.4711e-05,
          -5.4711e-05, -1.8237e-05],
         [-1.8237e-05, -1.8237e-05, -1.8237e-05,  ...,  1.8237e-05,
           1.8237e-05,  5.4711e-05]],

        [[-1.8237e-05,  5.4711e-05,  0.0000e+00,  ...,  0.0000e+00,
          -1.8237e-05,  5.4711e-05],
         [-1.8237e-05,  5.4711e-05,  0.0000e+00,  ...,  0.0000e+00,
           1.8237e-05, -5.4711e-05],
         [-1.8237e-05,  5.4711e-05,  0.0000e+00,  ...,  0.0000e+00,
           1.8237e-05, -5.4711e-05],
         ...,
         [-5.4711e-05,  1